In [1]:
import pandas as pd
pd.set_option('display.max_columns',500)

## SIEDU

In [3]:
! ls ../

PM25.nc
PM25ant.2014062600_2014062700_TICO2014.CL-CTROSUR_CR2-MMA.10K.nc
Resultados_Nicolas_Huneeus_20170124
Untitled.ipynb
basemap-1.1.0
basemap-1.1.0.tar.gz
calidad_aire_2050_cr2
datos
enviroment.yml
tutorial_NETCDF


In [4]:
import glob, os
os.chdir("/Users/pipe/Documents/Spike/CR2/datos/INDICADORES_SIEDU/")
siedu_files = {}
siedu_sheetnames = {}

#xl.sheet_names 
for file in glob.glob("*.xlsx"):
    nom_file = file.split('.')[0] 
    siedu_sheetnames[nom_file] = pd.ExcelFile('../INDICADORES_SIEDU/'+file)#, sheet_name=nom_file.split(' ')[0]+'_INDICADOR')
    print(f'nombre archivo: {nom_file}, sheets {siedu_sheetnames[nom_file].sheet_names}')
    for element in siedu_sheetnames[nom_file].sheet_names:
        siedu_files[element] = siedu_sheetnames[nom_file].parse(element)
#     print(nom_file, siedu[nom_file].sheet_names)

nombre archivo: IS_34 , sheets ['IS_34_METADATO', 'IS_34_INDICADOR']
nombre archivo: BPU_28, sheets ['BPU_28_METADATO', 'BPU_28_INDICADOR PLAZAS', 'BPU_28_INDICADOR PARQUES']
nombre archivo: EA_48, sheets ['EA_48_METADATO', 'EA_48_INDICADOR']
nombre archivo: IG_66, sheets ['IG_66_METADATO', 'IG_66_INDICADOR']
nombre archivo: IS_32 , sheets ['IS_32_METADATO', 'IS_32_INDICADOR']
nombre archivo: BPU_25, sheets ['BPU_25_METADATO', 'BPU_25_INDICADOR']
nombre archivo: EA_93 , sheets ['EA_93_METADATO', 'EA_93_INDICADOR']
nombre archivo: DE_3, sheets ['DE_3_METADATO', 'DE_3_INDICADOR']
nombre archivo: BPU_29, sheets ['BPU_29_METADATO', 'BPU_29_INDICADOR']
nombre archivo: BPU_4, sheets ['BPU_4_METADATA', 'BPU_4_INDICADOR']
nombre archivo: BPU_22, sheets ['BPU_22_METADATO', 'BPU_22_INDICADOR']
nombre archivo: BPU_3, sheets ['BPU_3_METADATO', 'BPU_3_INDICADOR']
nombre archivo: IP_47, sheets ['IP_47_METADATO', 'IP_47_INDICADOR']
nombre archivo: DE_36, sheets ['DE_36_METADATO', 'DE_36_INDICADOR']
n

In [5]:
lista_indicadores = [m for m in siedu_files.keys() if 'INDICADOR' in m or 'RESULTADOS' in m]

columnas_comun = ['Región', 'Nombre Ciudad', 'Cod_Ciudad', 'Provincia', 'Área', 'Comuna', 'CUT']

In [7]:
indicadores_raros = []
indicadores_buenis = []
for indicador in lista_indicadores:
    if set(columnas_comun).difference(set(siedu_files[indicador].columns)) == set():
        siedu_files[indicador].dropna(subset=columnas_comun, inplace=True)
        indicadores_buenis += [indicador]
    else:
        indicadores_raros += [indicador]


In [8]:
for indicador in indicadores_buenis:
    if 'Brecha' in siedu_files[indicador].columns:
        if 'INDICADOR' in indicador:
            tipo = indicador.split('_INDICADOR')[0]
            siedu_files[indicador].rename(columns={'Brecha':'Brecha_'+tipo}, inplace=True)

In [63]:
df_aux = siedu_files[indicadores_buenis[0]]
for indicador in indicadores_buenis[1:]:
    df_aux = pd.merge(df_aux, siedu_files[indicador], on=columnas_comun,how='left')
df_aux.drop(columns = [m for m in df_aux.columns if 'Unnamed' in m], inplace=True)

In [64]:
df_aux.head(3)

,Región,Nombre Ciudad,Cod_Ciudad,Provincia,Área,Comuna,CUT,IS_34 Porcentaje de viviendas con situación de allegamiento externo,Población total comunal LUC_x,Población usuaria de plazas,BPU_28 Porcentaje de población atendida por plazas públicas,Población total comunal LUC_y,Población usuaria de parques,BPU_28 Porcentaje de población atendida por parques públicas,Pob_Urb_2017,Pob_Urb_Afectada,EA_48 Porcentaje de población expuesta a inundación por tsunami,IG_66 Plan Regulador Comunal vigente,IS_32 Requerimiento de viviendas nuevas urbanas,BPU_25 Distancia a paraderos (m),Brecha_BPU_25,Longitud_Vias_M,Longitud_Vias_KM,Longitud_Ciclovias_M,Longitud_Ciclovias_KM,EA_93 Porcentaje de cobertura de la red de ciclovía sobre la red vial,DE_3 Porcentaje de participacion del Fondo Comun Municipal en el Ingreso Municipal Total,Brecha_DE_3,Superficie Parque (m2),Superficie Plazas (m2),Superficie de Area Verde (m2),Poblacion CPV 2017,BPU_29 Superficie de Area Verde por Habitantes (m2/Hab),Brecha_BPU_29,BPU_4 Razón entre disponibilidad efectiva de matrículas y demanda potencial por educación,Brecha_BPU_4,BPU_22 Distancia a parques públicos (m),Normalización,BPU_3 Distancia a E. Basica (m),Brecha_BPU_3,EA_22 Consumo Per Capita residencial (Kwh/persona),IS_31 Porcentaje de viviendas particulares que requieren mejoras de materialidad y/o servicios básicos,EA_34 Cantidad de Residuos (Kg/hab/dia),Brecha_EA_34,IS_40 Porcentaje de manzanas con veredas con buena calidad de pavimento,Brecha_IS_40,IS_33 Porcentaje de Hacinamiento,DE_16 Percentil 90 del tiempo de viaje en hora punta mañana,Brecha_DE_16,Población Área de Servicio_x,Superficie Parques M²,BPU_23 Superficie de parques por habitante,BPU_20 Distancia a Plazas Publicas (m),Brecha_BPU_20,BPU_1 Distancia a E. Inicial (m),Brecha_BPU_1,IS_39 Porcentaje de unidades vecinales que tienen entre 20% y 60% de hogares vulnerables,Brecha_IS_39,Población Área de Servicio_y,Superficie Plazas M²,BPU_21 Superficie de plazas por habitantes,IP_48 PRC reconoce inmuebles y/o zonas de conservación histórica,BPU_7 Distancia a Centro de Salud (m),Brecha_BPU_7,DE_29 Percentil 90 del tiempo de viaje en transporte público en hora punta mañana,Brecha_DE_29,Población 2017,Lesionados 2018,de_31 N° de lesionados en siniestros de tránsito por cada 100.000 habitantes,Poblacióm 2017,Fallecidos 2018,DE_28 N° de víctimas mortales en siniestros de tránsito por cada 100.000 habitantes,IP_43a Porcentaje de zonas típicas con lineamientos de intervención aprobados,IG_90 Porcentaje de participiación
0,TARAPACÁ,IQUIQUE – ALTO HOSPICIO,1001.0,IQUIQUE,OTRAS COMUNAS,IQUIQUE,1101.0,4.44,188003.0,128462.0,68.33,188003.0,154124.0,81.98,189065.0,56279.0,29.77,1,6949,248.34,Nula,412342,412.34,9415.25,9.42,2.28,8.24,Nula,132416,267101.90,399518.29,184793.0,2.16,Alta,0.93,Baja,2047.55,Nula,505.99,Nula,850.35,11.79,1.75,Alta,23.13,Alta,9.92,30,Nula,154124.0,132319.31,0.86,282.38,Nula,566.1,Baja,65.07,Alta,128462.0,267101.902455,2.07923,0,1044.58,Nula,35,Nula,191468.0,760.0,396.93,191468.0,10.0,5.22,100,29.65
1,TARAPACÁ,IQUIQUE – ALTO HOSPICIO,1001.0,IQUIQUE,OTRAS COMUNAS,ALTO HOSPICIO,1107.0,3.19,103807.0,71044.0,68.44,103807.0,85317.0,82.19,NaN,NaN,NaN,0,4978,268.64,Nula,309550,309.55,4831.87,4.83,1.56,56.5,Alta,100803,240669.45,341471.98,105700.0,3.23,Alta,0.91,Baja,1827.35,Nula,496.12,Nula,536.54,23.23,1.1,Baja,28.03,Alta,15.32,45,Nula,85317.0,100802.53,1.18,274.86,Nula,535.97,Baja,55.55,Alta,71044.0,240669.447216,3.38761,0,1067.96,Nula,60,Nula,108375.0,349.0,322.03,108375.0,4.0,3.69,S/I,29.09
2,ANTOFAGASTA,ANTOFAGASTA,2101.0,ANTOFAGASTA,OTRAS COMUNAS,ANTOFAGASTA,2101.0,4.34,347605.0,162827.0,46.84,347605.0,314744.0,90.55,354104.0,6127.0,1.73,1,13240,326.74,Nula,989856,989.86,27137.3,27.14,2.74,13.2,Nula,331807,470498.69,802305.20,349089.0,2.29,Alta,0.88,Media,1420.31,Nula,688.99,Nula,755.69,11.47,1.33,Alta,34.16,Alta,7.73,40,Nula,314744.0,331806.52,1.05,425.99,Baja,831.55,Media,71.79,Alta,162827.0,470491.396903,2.88952,1,1320.59,Nula,55,Nula

### Pasamos todos los campos a minuscula

In [65]:
df_aux.columns = df_aux.columns.str.lower()

for coli in ['región', 'nombre ciudad', 'provincia', 'área', 'comuna']:
    df_aux[coli] = df_aux[coli].str.lower()
df_aux.head(3)

,región,nombre ciudad,cod_ciudad,provincia,área,comuna,cut,is_34 porcentaje de viviendas con situación de allegamiento externo,población total comunal luc_x,población usuaria de plazas,bpu_28 porcentaje de población atendida por plazas públicas,población total comunal luc_y,población usuaria de parques,bpu_28 porcentaje de población atendida por parques públicas,pob_urb_2017,pob_urb_afectada,ea_48 porcentaje de población expuesta a inundación por tsunami,ig_66 plan regulador comunal vigente,is_32 requerimiento de viviendas nuevas urbanas,bpu_25 distancia a paraderos (m),brecha_bpu_25,longitud_vias_m,longitud_vias_km,longitud_ciclovias_m,longitud_ciclovias_km,ea_93 porcentaje de cobertura de la red de ciclovía sobre la red vial,de_3 porcentaje de participacion del fondo comun municipal en el ingreso municipal total,brecha_de_3,superficie parque (m2),superficie plazas (m2),superficie de area verde (m2),poblacion cpv 2017,bpu_29 superficie de area verde por habitantes (m2/hab),brecha_bpu_29,bpu_4 razón entre disponibilidad efectiva de matrículas y demanda potencial por educación,brecha_bpu_4,bpu_22 distancia a parques públicos (m),normalización,bpu_3 distancia a e. basica (m),brecha_bpu_3,ea_22 consumo per capita residencial (kwh/persona),is_31 porcentaje de viviendas particulares que requieren mejoras de materialidad y/o servicios básicos,ea_34 cantidad de residuos (kg/hab/dia),brecha_ea_34,is_40 porcentaje de manzanas con veredas con buena calidad de pavimento,brecha_is_40,is_33 porcentaje de hacinamiento,de_16 percentil 90 del tiempo de viaje en hora punta mañana,brecha_de_16,población área de servicio_x,superficie parques m²,bpu_23 superficie de parques por habitante,bpu_20 distancia a plazas publicas (m),brecha_bpu_20,bpu_1 distancia a e. inicial (m),brecha_bpu_1,is_39 porcentaje de unidades vecinales que tienen entre 20% y 60% de hogares vulnerables,brecha_is_39,población área de servicio_y,superficie plazas m²,bpu_21 superficie de plazas por habitantes,ip_48 prc reconoce inmuebles y/o zonas de conservación histórica,bpu_7 distancia a centro de salud (m),brecha_bpu_7,de_29 percentil 90 del tiempo de viaje en transporte público en hora punta mañana,brecha_de_29,población 2017,lesionados 2018,de_31 n° de lesionados en siniestros de tránsito por cada 100.000 habitantes,poblacióm 2017,fallecidos 2018,de_28 n° de víctimas mortales en siniestros de tránsito por cada 100.000 habitantes,ip_43a porcentaje de zonas típicas con lineamientos de intervención aprobados,ig_90 porcentaje de participiación
0,tarapacá,iquique – alto hospicio,1001.0,iquique,otras comunas,iquique,1101.0,4.44,188003.0,128462.0,68.33,188003.0,154124.0,81.98,189065.0,56279.0,29.77,1,6949,248.34,Nula,412342,412.34,9415.25,9.42,2.28,8.24,Nula,132416,267101.90,399518.29,184793.0,2.16,Alta,0.93,Baja,2047.55,Nula,505.99,Nula,850.35,11.79,1.75,Alta,23.13,Alta,9.92,30,Nula,154124.0,132319.31,0.86,282.38,Nula,566.1,Baja,65.07,Alta,128462.0,267101.902455,2.07923,0,1044.58,Nula,35,Nula,191468.0,760.0,396.93,191468.0,10.0,5.22,100,29.65
1,tarapacá,iquique – alto hospicio,1001.0,iquique,otras comunas,alto hospicio,1107.0,3.19,103807.0,71044.0,68.44,103807.0,85317.0,82.19,NaN,NaN,NaN,0,4978,268.64,Nula,309550,309.55,4831.87,4.83,1.56,56.5,Alta,100803,240669.45,341471.98,105700.0,3.23,Alta,0.91,Baja,1827.35,Nula,496.12,Nula,536.54,23.23,1.1,Baja,28.03,Alta,15.32,45,Nula,85317.0,100802.53,1.18,274.86,Nula,535.97,Baja,55.55,Alta,71044.0,240669.447216,3.38761,0,1067.96,Nula,60,Nula,108375.0,349.0,322.03,108375.0,4.0,3.69,S/I,29.09
2,antofagasta,antofagasta,2101.0,antofagasta,otras comunas,antofagasta,2101.0,4.34,347605.0,162827.0,46.84,347605.0,314744.0,90.55,354104.0,6127.0,1.73,1,13240,326.74,Nula,989856,989.86,27137.3,27.14,2.74,13.2,Nula,331807,470498.69,802305.20,349089.0,2.29,Alta,0.88,Media,1420.31,Nula,688.99,Nula,755.69,11.47,1.33,Alta,34.16,Alta,7.73,40,Nula,314744.0,331806.52,1.05,425.99,Baja,831.55,Media,71.79,Alta,162827.0,470491.396903,2.88952,1,1320.59,Nula,55,Nula

In [66]:
df = pd.read_csv('/Users/pipe/Documents/Spike/CR2/datos/Censo2017_16R_ManzanaEntidad_CSV/Censo2017_Manzanas.csv',
                sep=';')
numero_habitantes = df.groupby('COMUNA')['PERSONAS'].sum()
numero_habitantes = numero_habitantes.reset_index()
numero_habitantes = numero_habitantes.rename(columns={'COMUNA':'cut', 'PERSONAS': 'num_personas'})

In [67]:
df_aux['cut'] = df_aux['cut'].astype('int')

In [68]:
df_aux.shape[0]

117

In [69]:
numero_habitantes.shape[0]

346

In [70]:
df_aux = df_aux.merge(numero_habitantes, on='cut', how='right')

In [71]:
df_aux.shape[0]

346

In [72]:
df_aux.to_csv('/Users/pipe/Documents/Spike/CR2/calidad_aire_2050_cr2/datos_streamlit/socioeconomicos_por_comuna.csv', index=False)

In [74]:
df_aux.tail()

,región,nombre ciudad,cod_ciudad,provincia,área,comuna,cut,is_34 porcentaje de viviendas con situación de allegamiento externo,población total comunal luc_x,población usuaria de plazas,bpu_28 porcentaje de población atendida por plazas públicas,población total comunal luc_y,población usuaria de parques,bpu_28 porcentaje de población atendida por parques públicas,pob_urb_2017,pob_urb_afectada,ea_48 porcentaje de población expuesta a inundación por tsunami,ig_66 plan regulador comunal vigente,is_32 requerimiento de viviendas nuevas urbanas,bpu_25 distancia a paraderos (m),brecha_bpu_25,longitud_vias_m,longitud_vias_km,longitud_ciclovias_m,longitud_ciclovias_km,ea_93 porcentaje de cobertura de la red de ciclovía sobre la red vial,de_3 porcentaje de participacion del fondo comun municipal en el ingreso municipal total,brecha_de_3,superficie parque (m2),superficie plazas (m2),superficie de area verde (m2),poblacion cpv 2017,bpu_29 superficie de area verde por habitantes (m2/hab),brecha_bpu_29,bpu_4 razón entre disponibilidad efectiva de matrículas y demanda potencial por educación,brecha_bpu_4,bpu_22 distancia a parques públicos (m),normalización,bpu_3 distancia a e. basica (m),brecha_bpu_3,ea_22 consumo per capita residencial (kwh/persona),is_31 porcentaje de viviendas particulares que requieren mejoras de materialidad y/o servicios básicos,ea_34 cantidad de residuos (kg/hab/dia),brecha_ea_34,is_40 porcentaje de manzanas con veredas con buena calidad de pavimento,brecha_is_40,is_33 porcentaje de hacinamiento,de_16 percentil 90 del tiempo de viaje en hora punta mañana,brecha_de_16,población área de servicio_x,superficie parques m²,bpu_23 superficie de parques por habitante,bpu_20 distancia a plazas publicas (m),brecha_bpu_20,bpu_1 distancia a e. inicial (m),brecha_bpu_1,is_39 porcentaje de unidades vecinales que tienen entre 20% y 60% de hogares vulnerables,brecha_is_39,población área de servicio_y,superficie plazas m²,bpu_21 superficie de plazas por habitantes,ip_48 prc reconoce inmuebles y/o zonas de conservación histórica,bpu_7 distancia a centro de salud (m),brecha_bpu_7,de_29 percentil 90 del tiempo de viaje en transporte público en hora punta mañana,brecha_de_29,población 2017,lesionados 2018,de_31 n° de lesionados en siniestros de tránsito por cada 100.000 habitantes,poblacióm 2017,fallecidos 2018,de_28 n° de víctimas mortales en siniestros de tránsito por cada 100.000 habitantes,ip_43a porcentaje de zonas típicas con lineamientos de intervención aprobados,ig_90 porcentaje de participiación,num_personas
341,NaN,NaN,NaN,NaN,NaN,NaN,16207,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5401
342,NaN,NaN,NaN,NaN,NaN,NaN,16302,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26881
343,NaN,NaN,NaN,NaN,NaN,NaN,16303,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11152
344,NaN,NaN,NaN,NaN,NaN,NaN,16304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4308
345,NaN,NaN,NaN,NaN,NaN,NaN,16305,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [76]:
df_aux.comuna.unique()

array(['iquique', 'alto hospicio', 'antofagasta', 'calama', 'copiapó',
       'tierra amarilla', 'vallenar', 'la serena', 'coquimbo', 'ovalle',
       'valparaíso', 'casablanca', 'concón', 'puchuncaví', 'quintero',
       'viña del mar', 'los andes', 'san esteban', 'quillota', 'calera',
       'hijuelas', 'la cruz', 'san antonio', 'cartagena', 'santo domingo',
       'san felipe', 'quilpué', 'limache', 'olmué', 'villa alemana',
       'rancagua', 'machalí', 'rengo', 'san fernando', 'talca',
       'constitución', 'maule', 'curicó', 'rauco', 'romeral', 'linares',
       'concepción', 'coronel', 'chiguayante', 'hualqui', 'lota', 'penco',
       'san pedro de la paz', 'santa juana', 'talcahuano', 'tomé',
       'hualpén', 'los ángeles', 'nacimiento', 'temuco',
       'padre las casas', 'villarrica', 'angol', 'puerto montt',
       'puerto varas', 'castro', 'osorno', 'coyhaique', 'punta arenas',
       'santiago', 'cerrillos', 'cerro navia', 'conchalí', 'el bosque',
       'estación centra

In [51]:
df_aux.num_personas.isna().sum()

0

In [77]:
lista_variables_socio = [  'num_personas',  
  'is_34 porcentaje de viviendas con situación de allegamiento externo',
       'población total comunal luc_x', 'población usuaria de plazas',
       'bpu_28 porcentaje de población atendida por plazas públicas',
       'población total comunal luc_y', 'población usuaria de parques',
       'bpu_28 porcentaje de población atendida por parques públicas',
       'pob_urb_2017', 'pob_urb_afectada',
       'ea_48 porcentaje de población expuesta a inundación por tsunami',
       'superficie plazas (m2)', 'superficie de area verde (m2)',
       'poblacion cpv 2017',
       'bpu_29 superficie de area verde por habitantes (m2/hab)',
       'bpu_4 razón entre disponibilidad efectiva de matrículas y demanda potencial por educación',
       'bpu_3 distancia a e. basica (m)',
       'ea_22 consumo per capita residencial (kwh/persona)',
       'is_31 porcentaje de viviendas particulares que requieren mejoras de materialidad y/o servicios básicos',
       'is_40 porcentaje de manzanas con veredas con buena calidad de pavimento',
       'is_33 porcentaje de hacinamiento', 'población área de servicio_x',
       'superficie parques m²', 'bpu_23 superficie de parques por habitante',
       'bpu_20 distancia a plazas publicas (m)',
       'bpu_1 distancia a e. inicial (m)', 'población área de servicio_y',
       'superficie plazas m²', 'bpu_21 superficie de plazas por habitantes',
       'población 2017', 'lesionados 2018',
       'de_31 n° de lesionados en siniestros de tránsito por cada 100.000 habitantes',
       'poblacióm 2017', 'fallecidos 2018',
       'de_28 n° de víctimas mortales en siniestros de tránsito por cada 100.000 habitantes',
       'ig_90 porcentaje de participiación']

porcentajenan = {}
for var_socio in lista_variables_socio:
    porcentajenan[var_socio] = df_aux[var_socio].isna().sum()/df_aux.shape[0]

pd.DataFrame().from_dict(porcentajenan, orient='index')

### Rellenamos

In [16]:
import geopandas as gpd
comunas = gpd.read_file("/Users/pipe/Documents/Spike/CR2/datos/mapas_censo/Comunas/comunas.shp")
comunas.columns = [m.lower() for m in comunas.columns]
comunas = comunas.to_crs({'init': 'epsg:4326'})
comunas.head(2)

,objectid,shape_leng,dis_elec,cir_sena,cod_comuna,codregion,st_area_sh,st_length_,region,comuna,provincia,geometry
0,48,170038.624165,16,8,6204,6,9.685774e+08,206184.271675,Región del Libertador Bernardo O'Higgins,Marchigüe,Cardenal Caro,"POLYGON ((-71.80071 -34.20728, -71.79992 -34.2..."
1,29,125730.104795,15,8,6102,6,4.157446e+08,151911.576827,Región del Libertador Bernardo O'Higgins,Codegua,Cachapoal,"POLYGON ((-70.35293 -33.95263, -70.35243 -33.9..."


In [18]:
for col in ['comuna']:
    comunas[col] = comunas[col].str.lower()
comunas.head(2)

,objectid,shape_leng,dis_elec,cir_sena,cod_comuna,codregion,st_area_sh,st_length_,region,comuna,provincia,geometry
0,48,170038.624165,16,8,6204,6,9.685774e+08,206184.271675,Región del Libertador Bernardo O'Higgins,marchigüe,Cardenal Caro,"POLYGON ((-71.80071 -34.20728, -71.79992 -34.2..."
1,29,125730.104795,15,8,6102,6,4.157446e+08,151911.576827,Región del Libertador Bernardo O'Higgins,codegua,Cachapoal,"POLYGON ((-70.35293 -33.95263, -70.35243 -33.9..."


In [21]:
df = df_aux.merge(comunas[['geometry', 'comuna', 'codregion']], on='comuna', how='left')
df.head(1)

,región,nombre ciudad,cod_ciudad,provincia,área,comuna,cut,is_34 porcentaje de viviendas con situación de allegamiento externo,población total comunal luc_x,población usuaria de plazas,bpu_28 porcentaje de población atendida por plazas públicas,población total comunal luc_y,población usuaria de parques,bpu_28 porcentaje de población atendida por parques públicas,pob_urb_2017,pob_urb_afectada,ea_48 porcentaje de población expuesta a inundación por tsunami,ig_66 plan regulador comunal vigente,is_32 requerimiento de viviendas nuevas urbanas,bpu_25 distancia a paraderos (m),brecha_bpu_25,longitud_vias_m,longitud_vias_km,longitud_ciclovias_m,longitud_ciclovias_km,ea_93 porcentaje de cobertura de la red de ciclovía sobre la red vial,de_3 porcentaje de participacion del fondo comun municipal en el ingreso municipal total,brecha_de_3,superficie parque (m2),superficie plazas (m2),superficie de area verde (m2),poblacion cpv 2017,bpu_29 superficie de area verde por habitantes (m2/hab),brecha_bpu_29,bpu_4 razón entre disponibilidad efectiva de matrículas y demanda potencial por educación,brecha_bpu_4,bpu_22 distancia a parques públicos (m),normalización,bpu_3 distancia a e. basica (m),brecha_bpu_3,ea_22 consumo per capita residencial (kwh/persona),is_31 porcentaje de viviendas particulares que requieren mejoras de materialidad y/o servicios básicos,ea_34 cantidad de residuos (kg/hab/dia),brecha_ea_34,is_40 porcentaje de manzanas con veredas con buena calidad de pavimento,brecha_is_40,is_33 porcentaje de hacinamiento,de_16 percentil 90 del tiempo de viaje en hora punta mañana,brecha_de_16,población área de servicio_x,superficie parques m²,bpu_23 superficie de parques por habitante,bpu_20 distancia a plazas publicas (m),brecha_bpu_20,bpu_1 distancia a e. inicial (m),brecha_bpu_1,is_39 porcentaje de unidades vecinales que tienen entre 20% y 60% de hogares vulnerables,brecha_is_39,población área de servicio_y,superficie plazas m²,bpu_21 superficie de plazas por habitantes,ip_48 prc reconoce inmuebles y/o zonas de conservación histórica,bpu_7 distancia a centro de salud (m),brecha_bpu_7,de_29 percentil 90 del tiempo de viaje en transporte público en hora punta mañana,brecha_de_29,población 2017,lesionados 2018,de_31 n° de lesionados en siniestros de tránsito por cada 100.000 habitantes,poblacióm 2017,fallecidos 2018,de_28 n° de víctimas mortales en siniestros de tránsito por cada 100.000 habitantes,ip_43a porcentaje de zonas típicas con lineamientos de intervención aprobados,ig_90 porcentaje de participiación,geometry,codregion
0,tarapacá,iquique – alto hospicio,1001.0,iquique,otras comunas,iquique,1101.0,4.44,188003.0,128462.0,68.33,188003.0,154124.0,81.98,189065.0,56279.0,29.77,1,6949,248.34,Nula,412342,412.34,9415.25,9.42,2.28,8.24,Nula,132416,267101.9,399518.29,184793.0,2.16,Alta,0.93,Baja,2047.55,Nula,505.99,Nula,850.35,11.79,1.75,Alta,23.13,Alta,9.92,30,Nula,154124.0,132319.31,0.86,282.38,Nula,566.1,Baja,65.07,Alta,128462.0,267101.902455,2.07923,0,1044.58,Nula,35,Nula,191468.0,760.0,396.93,191468.0,10.0,5.22,100,29.65,"MULTIPOLYGON (((-70.16034 -20.21070, -70.16031...",1.0


In [23]:
lista_variables_numericas = df[list(df.columns)[7:-2]].fillna(0).dtypes[df[list(df.columns)[7:-2]].fillna(0).dtypes == 'float64'].index
lista_variables_numericas

Index(['is_34 porcentaje de viviendas con situación de allegamiento externo',
       'población total comunal luc_x', 'población usuaria de plazas',
       'bpu_28 porcentaje de población atendida por plazas públicas',
       'población total comunal luc_y', 'población usuaria de parques',
       'bpu_28 porcentaje de población atendida por parques públicas',
       'pob_urb_2017', 'pob_urb_afectada',
       'ea_48 porcentaje de población expuesta a inundación por tsunami',
       'superficie plazas (m2)', 'superficie de area verde (m2)',
       'poblacion cpv 2017',
       'bpu_29 superficie de area verde por habitantes (m2/hab)',
       'bpu_4 razón entre disponibilidad efectiva de matrículas y demanda potencial por educación',
       'bpu_3 distancia a e. basica (m)',
       'ea_22 consumo per capita residencial (kwh/persona)',
       'is_31 porcentaje de viviendas particulares que requieren mejoras de materialidad y/o servicios básicos',
       'is_40 porcentaje de manzanas con ver

In [107]:
gdf_to_json = gpd.GeoDataFrame(df, geometry='geometry')

In [113]:

gdf_to_json.isna().sum()[gdf_to_json.isna().sum() > 0]

pob_urb_2017                                                       94
pob_urb_afectada                                                   94
ea_48 porcentaje de población expuesta a inundación por tsunami    94
brecha_de_3                                                         1
brecha_de_16                                                       54
geometry                                                            1
codregion                                                           1
dtype: int64

In [108]:
import tqdm
import json

In [114]:
gdf_to_json.query('codregion==@reg')

,región,nombre ciudad,cod_ciudad,provincia,área,comuna,cut,is_34 porcentaje de viviendas con situación de allegamiento externo,población total comunal luc_x,población usuaria de plazas,bpu_28 porcentaje de población atendida por plazas públicas,población total comunal luc_y,población usuaria de parques,bpu_28 porcentaje de población atendida por parques públicas,pob_urb_2017,pob_urb_afectada,ea_48 porcentaje de población expuesta a inundación por tsunami,ig_66 plan regulador comunal vigente,is_32 requerimiento de viviendas nuevas urbanas,bpu_25 distancia a paraderos (m),brecha_bpu_25,longitud_vias_m,longitud_vias_km,longitud_ciclovias_m,longitud_ciclovias_km,ea_93 porcentaje de cobertura de la red de ciclovía sobre la red vial,de_3 porcentaje de participacion del fondo comun municipal en el ingreso municipal total,brecha_de_3,superficie parque (m2),superficie plazas (m2),superficie de area verde (m2),poblacion cpv 2017,bpu_29 superficie de area verde por habitantes (m2/hab),brecha_bpu_29,bpu_4 razón entre disponibilidad efectiva de matrículas y demanda potencial por educación,brecha_bpu_4,bpu_22 distancia a parques públicos (m),normalización,bpu_3 distancia a e. basica (m),brecha_bpu_3,ea_22 consumo per capita residencial (kwh/persona),is_31 porcentaje de viviendas particulares que requieren mejoras de materialidad y/o servicios básicos,ea_34 cantidad de residuos (kg/hab/dia),brecha_ea_34,is_40 porcentaje de manzanas con veredas con buena calidad de pavimento,brecha_is_40,is_33 porcentaje de hacinamiento,de_16 percentil 90 del tiempo de viaje en hora punta mañana,brecha_de_16,población área de servicio_x,superficie parques m²,bpu_23 superficie de parques por habitante,bpu_20 distancia a plazas publicas (m),brecha_bpu_20,bpu_1 distancia a e. inicial (m),brecha_bpu_1,is_39 porcentaje de unidades vecinales que tienen entre 20% y 60% de hogares vulnerables,brecha_is_39,población área de servicio_y,superficie plazas m²,bpu_21 superficie de plazas por habitantes,ip_48 prc reconoce inmuebles y/o zonas de conservación histórica,bpu_7 distancia a centro de salud (m),brecha_bpu_7,de_29 percentil 90 del tiempo de viaje en transporte público en hora punta mañana,brecha_de_29,población 2017,lesionados 2018,de_31 n° de lesionados en siniestros de tránsito por cada 100.000 habitantes,poblacióm 2017,fallecidos 2018,de_28 n° de víctimas mortales en siniestros de tránsito por cada 100.000 habitantes,ip_43a porcentaje de zonas típicas con lineamientos de intervención aprobados,ig_90 porcentaje de participiación,geometry,codregion
114,ñuble,chillán – chillán viejo,16101.0,diguillín,otras comunas,chillán,16101.0,1.37,168343.0,110943.0,65.9,168343.0,138877.0,82.5,NaN,NaN,NaN,1,8401,279.17,Nula,621300,621.3,11501.2,11.5,1.85,46.23,Media,51566.2,916944.40,968510.59,165887.0,5.83,Media,0.97,Baja,1674.6,Nula,767.29,Nula,762.08,11.53,1.07,Baja,58.84,Alta,5.43,S/I,NaN,138877.0,51566.02,0.37,232.25,Nula,684.32,Media,50,Alta,110943.0,916932.300868,8.2649,1,1111.43,Nula,S/I,S/I,184739.0,747.0,404.35,184739.0,14.0,7.58,0,33.79,"POLYGON ((-71.98706 -36.51001, -71.98604 -36.5...",16.0
115,ñuble,chillán – chillán viejo,16101.0,diguillín,otras comunas,chillán viejo,16103.0,0.73,27359.0,14047.0,51.34,27359.0,25902.0,94.67,NaN,NaN,NaN,1,8406,287.2,Nula,96736.7,96.74,5124.88,5.12,5.3,S/I,NaN,33932.1,103954.36,137886.44,27327.0,5.04,Media,0.67,Alta,1572.94,Nula,752.11,Nula,608.83,13.56,0.83,Nula,60.57,Alta,6.61,S/I,NaN,25902.0,33932.08,1.31,313.8,Nula,573.05,Baja,41.66,Alta,14047.0,103953.487137,7.4004,1,912.22,Nula,S/I,S/I,30907.0,132.0,427.09,30907.0,2.0,6.47,S/I,46.03,"POLYGON ((-72.35065 -36.69041, -72.35097 -36.6...",16.0
116,ñuble,san carlos,16301.0,punilla,otras comunas,san carlos,16301.0,0.81,33109.0,22101.0,66.75,33109.0,28774.0,86.91,NaN,NaN,NaN,1,8416,S/I,S/I,-,-,S/I,S/I,S/I,64.23,Alta,106992,146162.46,253154.70,32870.0,7.7,Baja,1.1,Nula,1353.43,Nula,543.63,Nula,604.74,16.64,0.75,Nula,39.13,Alta,6,S/I,NaN,28774.0,106992.26,3.72,288.51,Nula,535.19,Baja,S/I

In [116]:
'R'+str(int(reg))

'R16'

In [120]:
import numpy as np

In [126]:
gdf_to_json.query('codregion==13').comuna.sort_values().unique()

array(['buin', 'calera de tango', 'cerrillos', 'cerro navia', 'colina',
       'conchalí', 'el bosque', 'el monte', 'estación central',
       'huechuraba', 'independencia', 'isla de maipo', 'la cisterna',
       'la florida', 'la granja', 'la pintana', 'la reina', 'lampa',
       'las condes', 'lo barnechea', 'lo espejo', 'lo prado', 'macul',
       'maipú', 'melipilla', 'padre hurtado', 'paine',
       'pedro aguirre cerda', 'peñaflor', 'peñalolén', 'pirque',
       'providencia', 'pudahuel', 'puente alto', 'quilicura',
       'quinta normal', 'recoleta', 'renca', 'san bernardo',
       'san joaquín', 'san josé de maipo', 'san miguel', 'san ramón',
       'santiago', 'talagante', 'tiltil', 'vitacura', 'ñuñoa'],
      dtype=object)

In [121]:
regiones = df.codregion.unique()
datos_por_region_json = {}
for reg in np.arange(1,17):
        datos_por_region_json['R'+str(int(reg))] = json.loads(gdf_to_json.query('codregion==@reg').to_json())

for key in datos_por_region_json.keys():
    with open('/Users/pipe/Documents/Spike/CR2/calidad_aire_2050_cr2/datos_git/json_socioeconomicos_'+key+'.json', 'w') as f:
        json.dump(datos_por_region_json[key], f)

In [92]:
!pwd

/Users/pipe/Documents/Spike/CR2/datos/INDICADORES_SIEDU


In [148]:
df.dropna(subset=['codregion'], inplace=True)

In [149]:
df['codregion'] = df['codregion'].astype('int')

In [98]:
df_aux.columns = df_aux.columns.str.lower()

In [151]:
df_aux.to_csv('/Users/pipe/Documents/Spike/CR2/datos/calidad_aire_2050_cr2/datos_streamlit/indicadores_siedu_por_comuna.csv', index=False)

### PENDIENTE: seguir con los otros indicadores: indicadores_raros

# CENSO 2017 

# División del territorio:
## División político-administrativa
Pais > Región > Provincia > Comuna 

## División censal (límites censales definidos por el INE)
### Área urbana:
Comuna > Distrito Censal  > Zona > Manzana
### Área rural:
Comuna > Localidad > Entidad

In [2]:
! pwd

/Users/pipe/Documents/Spike/CR2/calidad_aire_2050_cr2


In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('/Users/pipe/Documents/Spike/CR2/datos/Censo2017_16R_ManzanaEntidad_CSV/Censo2017_Manzanas.csv',
                sep=';')
df.head(2)

,REGION,PROVINCIA,COMUNA,DC,AREA,ZC_LOC,MZ_ENT,ID_ZONA_LOC,ID_MANZENT,PERSONAS,...,MATREC,MATIRREC,P05_1,P05_2,P05_3,P05_4,REGION_15R,PROVINCIA_15R,COMUNA_15R,ID_MANZENT_15R
0,1,11,1101,1,1,1,1,7849,1101011001001,15,...,0,0,0,0,0,0,1,11,1101,1101011001001
1,1,11,1101,1,1,1,10,7849,1101011001010,70,...,1,0,15,0,0,0,1,11,1101,1101011001010


In [4]:
df.replace({'*':0}, inplace=True)

In [5]:
df['PUEBLO'] = df['PUEBLO'].astype('int')
df['PERSONAS'] = df['PERSONAS'].astype('int')
df['INMIGRANTES'] = df['INMIGRANTES'].astype('int')

In [6]:
numero_habitantes = df.groupby('COMUNA')['PERSONAS'].sum()

In [7]:
datos_socioec = df.groupby('COMUNA')[['PERSONAS','PUEBLO','INMIGRANTES']].agg({'PERSONAS':'sum', 'PUEBLO':'sum', 'INMIGRANTES':'sum',}).reset_index()
datos_socioec.head(2)

,COMUNA,PERSONAS,PUEBLO,INMIGRANTES
0,1101,191468,32705,26777
1,1107,108375,31976,12129


In [8]:
datos_socioec.to_csv('/Users/pipe/Documents/Spike/CR2/datos/calidad_aire_2050_cr2/datos_streamlit/indicadores_CENSO17_por_comuna.csv', index=False)

diccionario_columnas = {'REGION': 'código de cada región, de 1 a 16',
 'PROVINCIA': 'concatena región + número de provincia',
 'COMUNA': 'concatena región, provincia y comuna',
 'DC' : 'distrito censal' (99 = missing value),
 'AREA' : '1: urbana, 2: rural',
 'ZC_LOC' : 'zona censal' o 'localidad' (999=missingf_value),
 'MZ_ENT' : 'manzana' o 'entidad',
 'ID_ZONA_LOC' : 'el id de la zona o de la localidad',
 'ID_MANZENT' : 'el id de la manzana o la entidad',
 'PERSONAS' : 'cantidad de personas',
 'HOMBRES' : 'cantidad de hombres',
 'MUJERES' : 'cantidad de mujeres',
 'EDAD_xAy' : 'cantidad de gente con edad entre x e y',
 'INMIGRANTES' : 'Total de personas migrantes',
 'PUEBLO' : 'Total de personas que se consideran pertenecientes a un pueblo indígena u originario',
 'VIV_PART' : 'Total de viviendas particulares',
 'VIV_COL' : 'Total de viviendas colectivas',
 'VPOMP' : 'Total de viviendas particulares ocupadas con moradores presentes',
 'TOTAL_VIV' : 'Total de viviendas',
 'CANT_HOG' : 'cantidad de hogares por vivienda',
 'P01' : 'tipo de vivienda (1 casa, 2 depto, etc)'
 'P03A' : 'material muros exteriores'
 'P03B' : 'material cubierta techo'
 'P03C' : 'material construcción del piso'
 'P05' : 'origen del agua'
 'MATACEP' : 'Total de viviendas con materialidad aceptable',
 'MATREC' : 'Total de viviendas con materialidad aceptable',
 'MATIRREC' : 'total de viviendas con materialidad irrecuperable',
}
